In [2]:
import pandas as pd

In [3]:
df_2020= pd.read_csv (r'full_2020.csv')
df_2019 = pd.read_csv (r'full_2019.csv')
df_2018 = pd.read_csv (r'full_2018.csv')

In [28]:
full.head()

,Spot,date,time,range,height_ft,swell_dir,swell_angle,period_sec,wind_kmh,wind_dir,wind,wind_angle,temp_°c,Conditions,Condition_ratings
0,Nazare,2018-01-01,6am,20-30ft,15.5,WNW,292,15,6,S,Cross/Offshore,186,8,Good,4
1,Nazare,2018-01-01,12pm,18-28ft,14.5,WNW,292,15,14,W,Onshore,277,15,Good,4
2,Nazare,2018-01-01,6pm,18-26ft,13.5,WNW,293,14,6,WSW,Onshore,253,11,Good,4
3,Nazare,2018-02-01,6am,16-26ft,12.5,WNW,297,14,5,SSW,Cross-shore,207,10,Good,4
4,Nazare,2018-02-01,12pm,14-24ft,11.5,WNW,297,14,11,WSW,Onshore,258,16,Good,4


In [4]:
full = pd.concat([df_2018,df_2019,df_2020], ignore_index=True)

In [10]:
full["date"] = pd.to_datetime(full.date)

In [29]:
full.Conditions.value_counts()

Not recommended    9633
Good               9588
Epic               9036
Mediocre           3795
Choppy              828
Name: Conditions, dtype: int64

In [22]:
conditions_rank = {
    'Epic':5, 
    'Good':4, 
    'Mediocre':3, 
    'Choppy':2,
    'Not recommended':1
}

In [27]:
full["Condition_ratings"] = full.Conditions.map(conditions_rank)

In [42]:
def group_by (col):
    full_prueba = full.groupby(["date","Spot"])[f'{col}'].agg("mean")
    df1 = pd.DataFrame(full_prueba)
    df1 = df1.reset_index()
    return df1
    

In [44]:
height = group_by("height_ft")

In [49]:
height.head()

,date,Spot,height_ft
0,2018-01-01,Fronton,3.333333
1,2018-01-01,Itacoatiara,0.800000
2,2018-01-01,Mosca Point,3.666667
3,2018-01-01,Nazare,14.500000
4,2018-01-01,Padang-Padang,5.666667


In [46]:
wind_speed = group_by("wind_kmh")

In [48]:
wind_speed.head()

,date,Spot,wind_kmh
0,2018-01-01,Fronton,19.666667
1,2018-01-01,Itacoatiara,9.666667
2,2018-01-01,Mosca Point,19.666667
3,2018-01-01,Nazare,8.666667
4,2018-01-01,Padang-Padang,12.333333


In [50]:
rating = group_by("Condition_ratings")

In [51]:
rating.head()

,date,Spot,Condition_ratings
0,2018-01-01,Fronton,5.0
1,2018-01-01,Itacoatiara,1.0
2,2018-01-01,Mosca Point,5.0
3,2018-01-01,Nazare,4.0
4,2018-01-01,Padang-Padang,3.0


In [52]:
from statsmodels.tsa.stattools import adfuller

In [54]:
dickey_height = adfuller(height["height_ft"])
dickey_height

(-12.0357021851382,
 2.8066770743884147e-22,
 39,
 10555,
 {'1%': -3.430969695978145,
  '5%': -2.8618138703454576,
  '10%': -2.5669157760426144},
 47202.80708705075)

In [55]:
dickey_wind = adfuller(wind_speed["wind_kmh"])
dickey_wind

(-11.767096491621826,
 1.1123238916061825e-21,
 39,
 10555,
 {'1%': -3.430969695978145,
  '5%': -2.8618138703454576,
  '10%': -2.5669157760426144},
 67831.33577937541)

In [63]:
dickey_condition= adfuller(rating["Condition_ratings"])
dickey_condition

(-9.892108943382373,
 3.535486258522491e-17,
 39,
 10555,
 {'1%': -3.430969695978145,
  '5%': -2.8618138703454576,
  '10%': -2.5669157760426144},
 26799.743742309172)

In [65]:
adf, pvalue, usedlag, nobs, critical_values, icbest = dickey_height

In [66]:
adf

-12.0357021851382

In [67]:
critical_values

{'1%': -3.430969695978145,
 '5%': -2.8618138703454576,
 '10%': -2.5669157760426144}

In [68]:
adf, pvalue, usedlag, nobs, critical_values, icbest = dickey_wind

In [69]:
adf

-11.767096491621826

In [70]:
critical_values

{'1%': -3.430969695978145,
 '5%': -2.8618138703454576,
 '10%': -2.5669157760426144}

In [71]:
adf, pvalue, usedlag, nobs, critical_values, icbest = dickey_condition

In [72]:
adf

-9.892108943382373

In [73]:
critical_values

{'1%': -3.430969695978145,
 '5%': -2.8618138703454576,
 '10%': -2.5669157760426144}